## This notebook is intended for mining of an export from your favourite notebook application or emails exported from an account. 
### As an entrypoint you need to have a folder with exported files.
#### before starting below, activate your virtual env

In [ ]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from io import BytesIO
from PyPDF2 import PdfReader

load_dotenv(override=True)

In [ ]:
# Setup local AI service
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "What is 2 + 2?"}]
)

print(response.choices[0].message.content)

In [25]:
# Setup frontier model with paid API

# load_dotenv()
# api_key=os.getenv('OPENAI_API_KEY')

# if api_key and api_key[:8]=='sk-proj-':
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? ")

# MODEL='gpt-4o-mini'
# openai=OpenAI()

#### First remove files with no content

#### Then manage attachment names. They are in "...files"-directories and mostly have not meaningful names.

In [16]:
from pathlib import Path
import os
import glob

from bs4 import BeautifulSoup

import shutil

def remove_with_string(directory, contains_string):
    # Loop through all items in the directory
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        
        # Check if the item contains the string "(1)"
        if contains_string in item:
            if os.path.isfile(item_path):
                os.remove(item_path)
                print(f'Removed file: {item}')
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                print(f'Removed folder: {item}')


def remove_obsolete_files(directory):
    # Loop through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.html'):
            filepath = os.path.join(directory, filename)
            
            # Open and parse the HTML file
            with open(filepath, 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                
                # Check if the content "Untitled Attachment" is in the file
                if "Untitled Attachment" in soup.get_text():
                    os.remove(filepath)
                    print(f'Removed: {filename}')


def get_file_extension(file_name):
    # os.path.splitext jakaa tiedostonimen ja sen päätteen
    _, extension = os.path.splitext(file_name)
    return extension

# Use glob to find directories ending with 'files'
def update_names(root_dir):
        directories = glob.glob(os.path.join(root_dir, '**/*files'), recursive=True)

        # Iterate over the directories and rename files in subfolders
        for directory in directories:
            for root, _, files in os.walk(directory):
                # Get the parent directory
                new_directory=(directory.strip('.pdf files'))
                new_directory=(new_directory.strip(' files'))
                new_directory="".join(new_directory.split())
                
                for i, file in enumerate(files):
                    new_name = f"{new_directory}_{i+1}{get_file_extension(file)}"
                    os.rename(os.path.join(directory, file), os.path.join(directory, new_name))
                    

In [17]:
# Cleaning the files and folders
#remove_with_string(os.getenv('LOCAL_DIRECTORY')," files")
#remove_obsolete_files(os.getenv('LOCAL_DIRECTORY'))
#update_names(")

# read files 
#notebook = Directory('os.getenv('LOCAL_DIRECTORY')')
#print(notebook.files)
# read a pdf
#article = PDF_Article('os.getenv('LOCAL_DIRECTORY')/......pdf')
#print(article.text)
# read a html
#article = HTML_Article('os.getenv('LOCAL_DIRECTORY')/.......html')
#print(article.title)
#print(article.text)
#display(Markdown(article.text))



In [18]:
from pathlib import Path
import os
import glob

class Directory:
    """A utility class to represent directory containing files"""

    def __init__(self, path):

        # Define the root directory to start the search
        # Get a list of all files in the directory
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        print(files)
        self.files = files                   

In [71]:
# Create article class
class PDF_Article:
    def __init__(self, file_path):
        """
        Create this Article object from the given file using the PyPDF2 library
        """
        # Open the file in binary read mode
        with open(file_path, 'rb') as file:
            # Read the binary data
            binary_data = file.read()

        pdf_bytes = BytesIO(binary_data)
        reader = PdfReader(pdf_bytes)
    
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    
        self.text = text
        self.title = reader.metadata.get("/Title", "No title found")

class HTML_Article:
    def __init__(self, file_path, file_name):
        """
        Create this Article object from the given file using the Soup library
        """
        # Open the file in binary read mode
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')
        self.title = file_name
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [ ]:
print(len(notebook.files))

In [ ]:
# Here is a Python script that selects files from the given directory based on the presence of specific keywords or phrases in their names:
import os
import ast

def select_files(directory_path, keywords, extensions):
    selected_files = []

    for filename in os.listdir(directory_path):
        # Check if any keyword is present in the filename
        if any(keyword.lower() in filename.lower() for keyword in keywords):
            # Check if the file has one of the specified extensions
            if any(filename.lower().endswith(ext.lower()) for ext in extensions):
                selected_files.append(filename)

    return selected_files

def create_articles(directory_path,html_files):
    html_articles = []
    print(html_files)
    for file in html_files:
        #print(directory_path+file)
        html_articles.append(HTML_Article(directory_path+file, file))
    return html_articles

# Example usage
# Define the keywords/phrases and directory path
keywords = os.getenv('KEYWORDS1')
keywords_list = ast.literal_eval(keywords)
directory_path = os.getenv('LOCAL_DIRECTORY')
extensions = ['.htm', '.html']

# Initialize an empty list to store selected file names
selected_files = []

selected_files = select_files(directory_path, keywords_list, extensions)
selected_files.sort()
print("Selected files:", len(selected_files))
articles = create_articles(directory_path, selected_files)

for article in articles:
    print("article text:",article.text)



Use a call to AI model to decide which files are relevant.
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [ ]:
#print(articles[0].text)
system_prompt = "You are provided with a HTML article in Finnish language. Print the contents of file in markdown. \n"

def user_prompt_for(article):
    user_prompt = f"You are looking at an article titled {article.title}"
    user_prompt += "\nThe contents of this article is as follows; \n"
    user_prompt += article.text
    return user_prompt

def messages_for(article):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(article)}
    ]
print(messages_for(articles[0]))

### Time to call AI API!

In [45]:
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

def summarize(article):
    #website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(article)
    )
    return response.choices[0].message.content


In [ ]:
summarize(articles[0])

In [85]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(articles):
    #summary = summarize(article)
    for article in articles:
        content = article.title+"<br>"+article.text
        #print(content)
        display(Markdown(content))

In [ ]:
display_summary(articles)